# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [1]:
import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [2]:
def get_vocabulary(corpus):
    """
    Funkcja zwracająca listę unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    # YOUR CODE HERE
    result = set()
    for line in corpus:
        for word in line:
            result.add(word)
            
    return result

In [3]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [4]:
def initalize_translation_prob(corpus1, corpus2):
    # YOUR CODE HERE
    result = {}
    set_en = get_vocabulary(corpus1)
    set_es = get_vocabulary(corpus2)
    P = 1 / len(set_en)
    
    for slowo_en in set_en:
        for slowo_es in set_es:
            result[(slowo_es, slowo_en)] = P

    return result

translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [5]:
translation_prob

{('la', 'house'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('la', 'the'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333,
 ('verde', 'the'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('casa', 'green'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [6]:
import collections


def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem powinny być wartości oczekiwane dla zmiennej przypisań słów w zdaniach
    (reprezentacja dowolna, nie weryfikowana przez sprawdzarkę)
    """
    expected = {}

    for k, (en_line, es_line) in enumerate(zip(corpora1, corpora2)):
        for en_word in en_line:
            for es_word in es_line:
                expected[(k, es_word, en_word)] = translation_prob.get((es_word, en_word), 0)

    combined = collections.defaultdict(float)
    for (i, es_word, en_word), value in expected.items():
        combined[(i, en_word)] += value

    result = {(i, es_word, en_word): value / combined[(i, en_word)] for
              (i, es_word, en_word), value in expected.items()}
        
    return result


assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [7]:
assigment_expected_values

{(0, 'casa', 'green'): 0.5,
 (0, 'verde', 'green'): 0.5,
 (0, 'casa', 'house'): 0.5,
 (0, 'verde', 'house'): 0.5,
 (1, 'la', 'the'): 0.5,
 (1, 'casa', 'the'): 0.5,
 (1, 'la', 'house'): 0.5,
 (1, 'casa', 'house'): 0.5,
 (2, 'la', 'the'): 0.3333333333333333,
 (2, 'casa', 'the'): 0.3333333333333333,
 (2, 'verde', 'the'): 0.3333333333333333,
 (2, 'la', 'green'): 0.3333333333333333,
 (2, 'casa', 'green'): 0.3333333333333333,
 (2, 'verde', 'green'): 0.3333333333333333,
 (2, 'la', 'house'): 0.3333333333333333,
 (2, 'casa', 'house'): 0.3333333333333333,
 (2, 'verde', 'house'): 0.3333333333333333}

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [8]:
import collections


def calculate_maximization(corpora1, corpora2, assigment_expected_values):
    # YOUR CODE HERE
    p_es_en = collections.defaultdict(float)
    p_total = collections.defaultdict(float)

    for (k, es_word, en_word), p in assigment_expected_values.items():
        p_es_en[(es_word, en_word)] += p
        p_total[en_word] += p

    result = {(es_word, en_word): p / p_total[en_word] for (es_word, en_word), p in p_es_en.items()}
    return result

translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [9]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [10]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('casa', 'house'), 0.5584045584045584), (('verde', 'house'), 0.22079772079772084), (('la', 'house'), 0.22079772079772084)]
---
[(('casa', 'house'), 0.6638923177619094), (('verde', 'house'), 0.16805384111904528), (('la', 'house'), 0.16805384111904528)]
---
[(('casa', 'house'), 0.7532968646774504), (('verde', 'house'), 0.12335156766127481), (('la', 'house'), 0.12335156766127481)]
---
[(('casa', 'house'), 0.8239601969358895), (('verde', 'house'), 0.08801990153205524), (('la', 'house'), 0.08801990153205524)]
---
[(('casa', 'house'), 0.8769766282684489), (('verde', 'house'), 0.061511685865775545), (('la', 'house'), 0.061511685865775545)]
---
[(('casa', 'house'), 0.915296630096382), (('verde', 'house'), 0.042351684951809056), (('la', 'house'), 0.042351684951809056)]
---
[(('casa', 'house'), 0.94228242707854), (('verde', 'house'), 0.028858786460729976), (('la', 'house'), 0.028858786460729976)]
---
[(('casa', 'house'), 0.9609498992371662), (('verde', 'house'), 0.019525050381416956), (('la', 

Wywołaj algorytm EM na poniższym korpusie.

In [11]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [12]:
# YOUR CODE HERE
translation_prob = initalize_translation_prob(english2, polish)
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish, translation_prob)
    translation_prob = calculate_maximization(english2, polish, assigment_expected_values)

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [13]:
translation_prob

{('pies', 'the'): 0.3333333333333333,
 ('pies', 'dog'): 1.0,
 ('dom', 'the'): 0.6663411458333334,
 ('dom', 'house'): 0.99951171875,
 ('zielony', 'the'): 0.0003255208333333333,
 ('zielony', 'green'): 0.5,
 ('dom', 'green'): 0.5,
 ('zielony', 'house'): 0.00048828125}

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [14]:
# YOUR CODE HERE
polish2 = [["NULL"] + pl_line for pl_line in polish]
translation_prob = initalize_translation_prob(english2, polish2)
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish2, translation_prob)
    translation_prob = calculate_maximization(english2, polish2, assigment_expected_values)

In [15]:
translation_prob

{('NULL', 'the'): 0.9884935162981091,
 ('pies', 'the'): 3.509562053739688e-05,
 ('NULL', 'dog'): 0.5,
 ('pies', 'dog'): 0.5,
 ('dom', 'the'): 0.01146338484277804,
 ('NULL', 'house'): 0.49983723958333337,
 ('dom', 'house'): 0.49983723958333337,
 ('zielony', 'the'): 8.00323857550188e-06,
 ('NULL', 'green'): 0.3333333333333333,
 ('zielony', 'green'): 0.3333333333333333,
 ('dom', 'green'): 0.3333333333333333,
 ('zielony', 'house'): 0.00032552083333333337}

Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

W pierwszym korpusie równoległym mamy takie same długości zdań (a więc KAŻDY wyraz z jednego języka teoretycznie tłumaczy się na drugi język), w przypadku podanego wyżej korpusu równoległego języka polskiego pojawiały się wolne miejsca na token NULL.